# Predicting Parkinson's using EMD features and XGBoost

## Importing libraries 

In [1]:
from tqdm import tqdm # Imports `tqdm` to create a progress bad in Python

import pandas as pd # Imports the pandas library for tabular data
import numpy as np # Imports the numpy library for mathematical operations and linear algebra

import sklearn.metrics # Import metrics from sklearn
from sklearn.model_selection import KFold, RepeatedStratifiedKFold, GridSearchCV # Import functions for cross validation

import xgboost as xgb # Import XGBoost for gradient boosting

## Load Data

In [4]:
# Construct dataframe for analysis

# Triad 10 file locations
HCaddr = ['https://github.com/alicerueda/DDK-EMD-Feature-Extraction/raw/master/EMD%20Features/EMDDeltaEMD/EMDHCTriadFC10Per.csv']
PDaddr = ['https://github.com/alicerueda/DDK-EMD-Feature-Extraction/raw/master/EMD%20Features/EMDDeltaEMD/EMDPDTriadFC10Per.csv']

# Import the HC file as a Pandas DataFrame
dfHCOriginal = pd.read_csv(HCaddr[0])
# Drop rows with missing data
dfHCOriginal.dropna()
# Print the dimensions of the HC DataFrame
print('HC matrix dim: ', dfHCOriginal.shape)

HC matrix dim:  (50, 434)


In [6]:
dfHCOriginal.sample(5)

,subject,segmentNum,numIMFMean,OBW1Mean,OBW2Mean,OBW3Mean,OBW4Mean,OBW5Mean,OBW6Mean,OBW7Mean,...,deltaFCenter1Kurt,deltaFCenter2Kurt,deltaFCenter3Kurt,deltaFCenter4Kurt,deltaFCenter5Kurt,deltaFCenter6Kurt,deltaFCenter7Kurt,deltaFCenter8Kurt,deltaAmpKurt,deltaDurKurt
3,5,5,18.400000,5834.389918,2996.736955,1512.771899,921.499627,650.367109,435.002445,308.511409,...,-1.247197,-0.588006,-1.112773,-1.692116,-0.500388,-1.823599,-1.638865,-1.152393,-1.331687,-1.178321
42,48,6,17.333333,5412.090344,2665.079231,1474.969522,947.282707,631.544667,380.347247,238.449042,...,1.130712,-0.545427,0.862769,-0.261852,-1.548414,-0.148295,-0.018426,-1.479477,-0.471779,-0.603850
13,16,6,17.333333,2825.836823,2426.806060,1522.185325,1005.110188,707.870021,431.694623,309.299581,...,-1.339784,-0.929820,0.812100,0.429179,-0.004793,-0.842655,-1.072529,-1.295060,-0.405968,-0.130691
47,53,5,17.600000,3819.477472,1779.217728,1115.145679,762.209430,470.808312,408.358014,284.503543,...,-1.743892,-1.695751,-1.237521,-1.214040,-1.713509,-0.157778,-0.906216,-1.063423,-0.968133,-1.022169
49,57,14,17.214286,4377.131164,2441.016872,1456.015245,1018.415511,763.698318,499.817008,344.269749,...,3.912488,0.720842,-0.080348,-1.101134,-1.382647,-0.305909,0.142950,0.539650,-0.466455,2.079480


In [8]:
# Import the HCPD file as a Pandas DataFrame
dfPDOriginal = pd.read_csv(PDaddr[0])
# Drop rows with missing data
dfPDOriginal.dropna()
# Print the dimensions of the PD DataFrame
print('PD matrix dim: ',dfPDOriginal.shape)

PD matrix dim:  (50, 434)


In [9]:
dfPDOriginal.sample(5)

,subject,segmentNum,numIMFMean,OBW1Mean,OBW2Mean,OBW3Mean,OBW4Mean,OBW5Mean,OBW6Mean,OBW7Mean,...,deltaFCenter1Kurt,deltaFCenter2Kurt,deltaFCenter3Kurt,deltaFCenter4Kurt,deltaFCenter5Kurt,deltaFCenter6Kurt,deltaFCenter7Kurt,deltaFCenter8Kurt,deltaAmpKurt,deltaDurKurt
19,24,5,17.600000,7540.817099,4367.079322,2572.086267,1116.582318,629.359546,400.253823,276.246821,...,-0.596557,-0.802874,-0.494195,-0.886024,-1.371501,-0.428244,-0.597491,-1.089430,-0.228105,-0.971258
13,16,14,17.642857,7092.316812,3236.344755,1732.383209,1151.991866,719.559378,435.625122,272.169407,...,3.929925,-0.811370,0.454821,2.319525,-0.831646,-0.218017,-0.084549,-0.541758,-0.844609,-0.149189
31,39,2,17.500000,4637.859215,1545.626928,957.296017,725.832829,423.754947,236.037141,155.298812,...,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000,-2.000000
40,50,9,17.666667,6891.517857,2903.671301,1483.651941,1003.472328,642.491005,354.929153,293.277285,...,1.455191,-1.475899,-1.467022,-1.348513,-1.128970,-0.368917,0.275739,0.724511,-0.376251,-0.645273
5,7,10,18.700000,6891.538538,3038.206970,1621.950853,1057.156618,711.403791,451.174315,304.063248,...,-0.493360,-1.047576,-0.628793,0.250853,0.471669,0.464322,-1.408225,-0.739158,-0.915768,-0.641562


This function removes rows from a DataFrame where all standardized (Std) feature values are zero.

In [13]:
# A utility function that removes rows that contain 0 for all std features
def removeZeroData(dftmp):
  colnames = dftmp.columns ## get all columns names
  stdcol = [header for header in colnames if 'Std' in header] ## filter column names that contain 'Std'
  stdtotal = dftmp[stdcol].sum(axis=1) ## select only the standardized features and calculate the sum of each row
  dropIdx = [i for i,std in enumerate(stdtotal) if std==0] ## create a list that has indixes of the standaridized that are == 0
  #print(stdtotal)
  print('Removing ', len(dropIdx), ' rows')
  for i in dropIdx: ## Iterates through the list of indices and drops each row where the sum of standardized features is zero
    dftmp = dftmp.drop(dftmp.index[i])
  return dftmp


Remove rows that contain 0 for all std features from the PD DataFrame

In [16]:
dfPDOriginal = removeZeroData(dfPDOriginal)
# Get the shape of the PD DataFrame
pdrow, pdcol = dfPDOriginal.shape
# Print the shape of the PD DataFrame
print(dfPDOriginal.shape)

Removing  0  rows
(50, 434)


Remove rows that contain 0 for all std features from the HC DataFrame

In [19]:
dfHCOriginal = removeZeroData(dfHCOriginal)
# Get the shape of the HC DataFrame
hcrow, hccol = dfHCOriginal.shape
# Print the shape of the HC DataFrame
print(dfHCOriginal.shape)

Removing  0  rows
(50, 434)


Combine the HC and PD DataFrames

In [22]:
dfHCPD = pd.concat([dfHCOriginal,dfPDOriginal], axis=0)
# Create labels. Assign 1 to HC rows and zeroes to PD rows
hclabels = np.ones(hcrow, dtype=int)
pdlabels = np.zeros(pdrow, dtype=int)

Append the PD labels to the HC labels.

In [24]:
#   Note that this is consistent with how the HC and PD DataFrames were combined:
#   there, the HC and PD DataFrames were concatenated in that order
Labels = np.reshape(np.append(hclabels, pdlabels), (-1,1)) # Label for HC=1 and PD=0
# Print the shapes of the HC and PD labels
print(hcrow,pdrow)
# Print the shape of the concatenated HC and PD label object
print('Label size', Labels.shape)

50 50
Label size (100, 1)


In [26]:
# Get the column names
colnames = dfHCPD.columns
# Get the features from the concatenated HC PD DataFrame as a Numpy array
rescaledfeat = np.array(dfHCPD.iloc[:,1:].values)

# Print the column names
print('Features: ', colnames)
# Print the dimensions of the HC PD DataFrame
print('dfHCPD matrix dim: ', dfHCPD.shape)
# Print the dimensions of the feature array (one less than the HC PD DataFrame)
print('size of HCPD features: ', rescaledfeat.shape)

Features:  Index(['subject', 'segmentNum', 'numIMFMean', 'OBW1Mean', 'OBW2Mean',
       'OBW3Mean', 'OBW4Mean', 'OBW5Mean', 'OBW6Mean', 'OBW7Mean',
       ...
       'deltaFCenter1Kurt', 'deltaFCenter2Kurt', 'deltaFCenter3Kurt',
       'deltaFCenter4Kurt', 'deltaFCenter5Kurt', 'deltaFCenter6Kurt',
       'deltaFCenter7Kurt', 'deltaFCenter8Kurt', 'deltaAmpKurt',
       'deltaDurKurt'],
      dtype='object', length=434)
dfHCPD matrix dim:  (100, 434)
size of HCPD features:  (100, 433)


## Create the cross validator for the hyperparameter search

- The following line creates a cross validator with 4 repeated stratified K-Folds, each with 5 splits. These folds are going to be used to find optimal hyperparameters.

- Stratified K-Folds differ from K-Folds by ensuring that the proportion of classes in each training and testing set is roughly the same as in the entire training set.

- Moreover, 4 repeated stratified K-Fold sets, each of 5 folds, are created, for a total of 20 folds. 

In [28]:
# Create the folds for the grid search cross validation
kf = RepeatedStratifiedKFold(n_splits=5, n_repeats=4, random_state=42)

## Load the classifier

In [30]:
# Load the classifier. In this case, we will use XGBoost's gradient boost classifier
cls = xgb.XGBClassifier()

## Create the hyperparameter search space

In [51]:
# Create a dict with the parameter values.
# The GridSearch algorithm will iterate through all possible combinations of these values
all_params_search = {
    'n_estimators': [100, 250],  # Number of boosting rounds (trees). Higher values increase model complexity.
    
    'booster': ['gbtree'],  # Type of booster to use. 'gbtree' means using decision trees.

    'lambda': [3, 4],  # L2 regularization term (Ridge regression). Helps prevent overfitting.

    'alpha': [0],  # L1 regularization term (Lasso regression). Helps with feature selection.

    'subsample': [0.95, 1],  # Fraction of samples used per boosting round. Reduces overfitting (1 means using all samples).

    'colsample_bytree': [0.015, 0.03],  # Fraction of features used per tree. Low values prevent overfitting.

    'max_depth': [8],  # Maximum depth of each tree. Controls model complexity (higher values increase depth).

    'min_child_weight': [2],  # Minimum sum of instance weights needed in a child node. Higher values prevent overfitting.

    'eta': [0.07],  # Learning rate (step size shrinkage). Lower values make training slower but more stable.

    'gamma': [0],  # Minimum loss reduction needed to make a further partition on a leaf node. Higher values reduce overfitting.

    'grow_policy': ['lossguide'],  # Tree growing policy. 'lossguide' grows the tree depth-wise (suitable for large datasets).

    'device': ['cpu'],  # Device to run XGBoost on (can be 'gpu' if you have GPU support).

    'verbosity': [0],  # Logging level (0 = silent, 1 = warnings, 2 = info, 3 = debug).

    'objective': ['binary:logistic'],  # Objective function for binary classification (outputs probability between 0 and 1).

    'random_state': [42],  # Random seed for reproducibility.

    'tree_method': ['hist'],  # Tree construction algorithm. 'hist' is optimized for speed and memory efficiency.

    'max_bin': [32]  # Number of bins used for feature discretization in histogram-based tree method (affects speed/accuracy).
}


## Create the grid search cross validation object

The following line creates the grid search cross validation object. This object accepts several parameters:
- The classifier (in this case, `cls`, our XGBoost classifier)
- The hyperparameter search space dictionary (`all_params_search`)
- The cross validator that creates train-validation splits (`kf`)
- A verbosity level (set to `1` here to suppress unnecessary output)
- A scoring metric (set to `roc_auc` to use the area under the ROC curve)

In [42]:
# Create the grid search cross validation object
clf = GridSearchCV(cls, all_params_search, cv=kf, verbose=1, scoring='roc_auc')
clf

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=4, n_splits=5, random_state=42),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importa...
             param_grid={'alpha': [0], 'booster': ['gbtree'],
                         'colsample_bytree': [0.015, 0.03], 'device': ['cpu'],
                         'eta': [0.07], 'gamma': [0],
                         'grow_policy': ['lossguide'], 'lambda': [3, 4],
                         'max_bin': [32], 'max_depth': [8],
                         'min_child_weight': [2], 'n_estimators': [100, 250],
                         'objective': ['binary:logistic'], 'random_state': [42],
                         'subsample': [0.95, 1], 'tree_method': ['hist'],
                         'verbosity': [0]},
             scoring='roc_auc', verbose=1)

## Perform grid search

In [44]:
# Fit the grid search cross validation object. This is the line that takes a lot of time!
clf.fit(rescaledfeat, Labels)

Fitting 20 folds for each of 16 candidates, totalling 320 fits


GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=4, n_splits=5, random_state=42),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importa...
             param_grid={'alpha': [0], 'booster': ['gbtree'],
                         'colsample_bytree': [0.015, 0.03], 'device': ['cpu'],
                         'eta': [0.07], 'gamma': [0],
                         'grow_policy': ['lossguide'], 'lambda': [3, 4],
                         'max_bin': [32], 'max_depth': [8],
                         'min_child_weight': [2], 'n_estimators': [100, 250],
                         'objective': ['binary:logistic'], 'random_state': [42],
                         'subsample': [0.95, 1], 'tree_method': ['hist'],
                         'verbosity': [0]},
             scoring='roc_auc', verbose=1)

The following cell retrieves the best scoring parameters from all possible combinations in the grid search:

In [46]:
# Get the best parameters from the grid search cross validation object.
clf.best_params_

{'alpha': 0,
 'booster': 'gbtree',
 'colsample_bytree': 0.015,
 'device': 'cpu',
 'eta': 0.07,
 'gamma': 0,
 'grow_policy': 'lossguide',
 'lambda': 4,
 'max_bin': 32,
 'max_depth': 8,
 'min_child_weight': 2,
 'n_estimators': 250,
 'objective': 'binary:logistic',
 'random_state': 42,
 'subsample': 0.95,
 'tree_method': 'hist',
 'verbosity': 0}

The following cel retrieves the best ROC AUC average validation score obtained for the above set of parameters:

In [49]:
# Get the best score from the grid search cross validation object.
clf.best_score_

0.8195

# Verify performance using 200 random cross validation splits

Having identified the optimal combination of parameters (`clf.best_params_`), these parameters are valuated if they perform well on a numerous amount of random cross-validation splits. The following cell defines a loop which creates 200 repeats of K-Fold cross-validation splits, with each of the 200 repeats having 10 folds. For each of the 200 repeats, the average and 10-fold standard deviation of the following metrics are recorded:
- ROC AUC
- accuracy
- F1 score
- precision
- recall


However, because we are using K-Fold as opposed to stratified K-Fold to split the data, there is a risk that we may end up with a bad split that does not contain any examples of one of the two classes in the train or test data; this can occur by random chance. Consequently, our `for` loop contains an inner `while` loop which checks for this condition and discards splits where the train and test do not contain both labels, repeating the iteration until 200 valid repeats are created.

Note that running the code below necessitates the fitting of 200*10=2000 models (one for each repeat and split) which is somewhat time consuming.


1. Perform 200 repeated K-Fold cross-validation (10 splits per repeat).
2. Ensure each split has both classes in the dataset.
3. Train an XGBoost model on each training split.
4. Evaluate performance on five metrics (AUC, accuracy, F1-score, precision, recall).
5. Store the mean and standard deviation of the metrics across 200 runs.

In [72]:
# Define some lists to store metric means
all_aucs = []
all_accuracies = []
all_f1s = []
all_precisions = []
all_recalls = []

# Define some lists to store metric standard deviations
all_aucs_std = []
all_accuracies_std = []
all_f1s_std = []
all_precisions_std = []
all_recalls_std = []

# Define the for loop
for i in tqdm(range(200)): # We have 200 repeats
    badsplit = True # initialize badplit to True
    while badsplit: # A loop that runs forever as long as there is a badsplit
        kf = KFold(n_splits=10, shuffle=True) # Create a random K-Fold cross validator with shuffle=True
        splits = list(kf.split(rescaledfeat)) # Split the data and store the splits in a list
        badsplits = [] # Initialize a list to store badsplits
        for j in splits: # A loop that populates badsplits with False or True
            if len(np.unique(Labels[j[1]])) == 2:
                badsplits.append(False) # If there are 2 labels, add False to badsplits
            else:
                badsplits.append(True) # If there aren't 2 labels, add True to badsplts
        badsplit = max(badsplits) # Set badsplit to True if any element in badsplits is True, otherwise False
    # Define some lists to store metrics for this repeat
    aucs = []
    accuracies = []
    f1s = []
    precisions = []
    recalls = []
    # Define a loop to process every one of the 10 CV splits in this repeat
    for s in splits:
        cls = xgb.XGBClassifier(**clf.best_params_) # Define classifier with best params from grid search
        cls.fit(rescaledfeat[s[0]], Labels[s[0]]) # Fit classifier to training data in the split
        preds_proba = cls.predict_proba(rescaledfeat[s[1]])[:,1] # Get the predicted probabilities of each class
        pred_labels = np.where(preds_proba > 0.5, 1, 0) # Assign class labels from probabilities using threshold
        # Calculate metrics and append to lists for this repeat
        auc = sklearn.metrics.roc_auc_score(Labels[s[1]], pred_labels)
        aucs.append(auc)
        accuracy = sklearn.metrics.accuracy_score(Labels[s[1]], pred_labels)
        accuracies.append(accuracy)
        f1 = sklearn.metrics.f1_score(Labels[s[1]], pred_labels)
        f1s.append(f1)
        precision = sklearn.metrics.precision_score(Labels[s[1]], pred_labels)
        precisions.append(precision)
        recall = sklearn.metrics.recall_score(Labels[s[1]], pred_labels)
        recalls.append(recall)
    # Once all splits in this repeat are processed, append the means and standard deviations to global lists
    all_aucs.append(np.mean(aucs))
    all_accuracies.append(np.mean(accuracies))
    all_f1s.append(np.mean(f1s))
    all_precisions.append(np.mean(precisions))
    all_recalls.append(np.mean(recalls))
    all_aucs_std.append(np.std(aucs))
    all_accuracies_std.append(np.std(accuracies))
    all_f1s_std.append(np.std(f1s))
    all_precisions_std.append(np.std(precisions))
    all_recalls_std.append(np.std(recalls))

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [13:00<00:00,  3.90s/it]


In [79]:
print(f'auc: {np.mean(all_aucs) * 100:.2f}%, {np.mean(all_aucs_std) * 100:.2f}%')
print(f'accuracy: {np.mean(all_accuracies) * 100:.2f}%, {np.mean(all_accuracies_std) * 100:.2f}%')
print(f'f1: {np.mean(all_f1s) * 100:.2f}%, {np.mean(all_f1s_std) * 100:.2f}%')
print(f'precision: {np.mean(all_precisions) * 100:.2f}%, {np.mean(all_precisions_std) * 100:.2f}%')
print(f'recall: {np.mean(all_recalls) * 100:.2f}%, {np.mean(all_recalls_std) * 100:.2f}%')

auc: 73.73%, 13.36%
accuracy: 72.73%, 13.13%
f1: 72.59%, 14.90%
precision: 71.51%, 19.69%
recall: 78.95%, 18.18%


As expected, the performance of the XGBoost model on this small dataset was very good. Note that there were some implementation differences between the way we coded XGBoost here and the original code for the paper, which you can find here. Nonetheless, it is not expected that these implementation differences are the cause of the discrepancy in the performance of the published SVM results and the XGBoost results that you produced here; rather, the observed differences are likely caused by the usage of a different model.

As was stated at the beginning of the lab, the results presented herein do not invalidate the results of the paper. If anything, they strengthen the case that the features proposed in the paper are predictive of Parkinson's disease.

## SVM

SVM (Support Vector Machine) is a supervised learning algorithm used for classification and regression tasks. It is best known for binary classification, where it tries to find the optimal decision boundary that separates data points into different classes.

**How SVM Works for Classification**
SVM finds a **hyperplane** that best separates the classes in your dataset. A **hyperplane** is just a line (in 2D), a plane (in 3D), or a higher-dimensional surface that divides data into two groups.

**Find the Best Hyperplane**
- The goal is to **maximize the margin** (distance) between the nearest points of each class and the hyperplane.
- The nearest points are called **support vectors**, and they determine the boundary.

**Maximize the Margin**
- A larger margin means **better generalization** (less overfitting).
- If multiple hyperplanes exist, SVM chooses the one with the **largest margin**.

**Handle Overlapping Data with Kernels**
- If data is **not linearly separable** (i.e., a straight line won’t work), SVM uses **kernels** to transform the data into a higher-dimensional space where it becomes separable.
- Common kernel functions:
  - **Linear Kernel** → Works when data is separable with a straight line.
  - **Polynomial Kernel** → Adds polynomial features to make data separable.
  - **RBF (Radial Basis Function) Kernel** → Maps data to infinite dimensions for better separation.

In [64]:
from sklearn.svm import SVC

In [81]:
# Define some lists to store metric means
all_aucs = []
all_accuracies = []
all_f1s = []
all_precisions = []
all_recalls = []

# Define some lists to store metric standard deviations
all_aucs_std = []
all_accuracies_std = []
all_f1s_std = []
all_precisions_std = []
all_recalls_std = []

# Define the for loop
for i in tqdm(range(200)): # We have 200 repeats
    badsplit = True # initialize badplit to True
    while badsplit: # A loop that runs forever as long as there is a badsplit
        kf = KFold(n_splits=10, shuffle=True) # Create a random K-Fold cross validator with shuffle=True
        splits = list(kf.split(rescaledfeat)) # Split the data and store the splits in a list
        badsplits = [] # Initialize a list to store badsplits
        for j in splits: # A loop that populates badsplits with False or True
            if len(np.unique(Labels[j[1]])) == 2:
                badsplits.append(False) # If there are 2 labels, add False to badsplits
            else:
                badsplits.append(True) # If there aren't 2 labels, add True to badsplts
        badsplit = max(badsplits) # Set badsplit to True if any element in badsplits is True, otherwise False
    # Define some lists to store metrics for this repeat
    aucs = []
    accuracies = []
    f1s = []
    precisions = []
    recalls = []
    # Define a loop to process every one of the 10 CV splits in this repeat
    for s in splits:
        # To use SVM, change the classifier:
        cls = SVC(probability=True) # Define classifier with best params from grid search
        # SVC's syntax is similar to XGBoost, you just need to reshape Labels with ravel or you will get warnings
        cls.fit(rescaledfeat[s[0]], Labels[s[0]].ravel()) # Fit classifier to training data in the split
        preds_proba = cls.predict_proba(rescaledfeat[s[1]])[:,1] # Get the predicted probabilities of each class
        pred_labels = np.where(preds_proba > 0.5, 1, 0) # Assign class labels from probabilities using threshold
        # Calculate metrics and append to lists for this repeat
        auc = sklearn.metrics.roc_auc_score(Labels[s[1]], pred_labels)
        aucs.append(auc)
        accuracy = sklearn.metrics.accuracy_score(Labels[s[1]], pred_labels)
        accuracies.append(accuracy)
        f1 = sklearn.metrics.f1_score(Labels[s[1]], pred_labels)
        f1s.append(f1)
        precision = sklearn.metrics.precision_score(Labels[s[1]], pred_labels,zero_division=0)
        precisions.append(precision)
        recall = sklearn.metrics.recall_score(Labels[s[1]], pred_labels)
        recalls.append(recall)
    # Once all splits in this repeat are processed, append the means and standard deviations to global lists
    all_aucs.append(np.mean(aucs))
    all_accuracies.append(np.mean(accuracies))
    all_f1s.append(np.mean(f1s))
    all_precisions.append(np.mean(precisions))
    all_recalls.append(np.mean(recalls))
    all_aucs_std.append(np.std(aucs))
    all_accuracies_std.append(np.std(accuracies))
    all_f1s_std.append(np.std(f1s))
    all_precisions_std.append(np.std(precisions))
    all_recalls_std.append(np.std(recalls))

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [00:43<00:00,  4.61it/s]


In [83]:
print(f'auc: {np.mean(all_aucs) * 100:.2f}%, {np.mean(all_aucs_std) * 100:.2f}%')
print(f'accuracy: {np.mean(all_accuracies) * 100:.2f}%, {np.mean(all_accuracies_std) * 100:.2f}%')
print(f'f1: {np.mean(all_f1s) * 100:.2f}%, {np.mean(all_f1s_std) * 100:.2f}%')
print(f'precision: {np.mean(all_precisions) * 100:.2f}%, {np.mean(all_precisions_std) * 100:.2f}%')
print(f'recall: {np.mean(all_recalls) * 100:.2f}%, {np.mean(all_recalls_std) * 100:.2f}%')

auc: 43.67%, 12.66%
accuracy: 38.38%, 12.65%
f1: 31.64%, 19.31%
precision: 34.92%, 24.23%
recall: 38.15%, 29.47%


## Better hyperparameter search using optuna

In machine learning, hyperparameter tuning is an important task where the model parameters are optimized to improve performance. Optuna is a powerful optimization framework that automates hyperparameter tuning.

In [85]:
import optuna

In [87]:
splits = list(kf.split(rescaledfeat, Labels))

This function is the core of Optuna’s hyperparameter optimization process. The trial argument represents a single trial (iteration) in the optimization process, where Optuna suggests a set of hyperparameters for the model.

In [89]:
def objective(trial):
    param = {
        "device": "cuda",
        "verbosity": 0,
        "objective": "binary:logistic",
        "random_state": 42,
        # use exact for small dataset.
        "tree_method": "hist",
        "max_bin": 32,
        "n_estimators": trial.suggest_int("n_estimators", 50, 350, log=True),
        # defines booster, gblinear for linear functions.
        "booster": trial.suggest_categorical("booster", ["gbtree", "dart"]),
        # L2 regularization weight.
        "lambda": trial.suggest_float("lambda", 1.0, 10, log=True),
        # L1 regularization weight.
        "alpha": trial.suggest_float("alpha", 1e-8, 10, log=True),
        # sampling ratio for training data.
        "subsample": trial.suggest_float("subsample", 0.2, 1.0, log=True),
        # sampling according to each tree.
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.01, 0.1, log=True),
    }

    if param["booster"] in ["gbtree", "dart"]:
        # maximum depth of the tree, signifies complexity of the tree.
        param["max_depth"] = trial.suggest_int("max_depth", 1, 12, log=True)
        # minimum child weight, larger the term more conservative the tree.
        param["min_child_weight"] = trial.suggest_int("min_child_weight", 1, 10, log=True)
        param["eta"] = trial.suggest_float("eta", 1e-8, 1.0, log=True)
        # defines how selective algorithm is.
        param["gamma"] = trial.suggest_float("gamma", 1e-8, 1.0, log=True)
        param["grow_policy"] = trial.suggest_categorical("grow_policy", ["depthwise", "lossguide"])

    if param["booster"] == "dart":
        param["sample_type"] = trial.suggest_categorical("sample_type", ["uniform", "weighted"])
        param["normalize_type"] = trial.suggest_categorical("normalize_type", ["tree", "forest"])
        param["rate_drop"] = trial.suggest_float("rate_drop", 1e-8, 1.0, log=True)
        param["skip_drop"] = trial.suggest_float("skip_drop", 1e-8, 1.0, log=True)

    mymetrics = []
    for s in splits:
        cls = xgb.XGBClassifier(**param)
        cls.fit(rescaledfeat[s[0]], Labels[s[0]])
        preds_proba = cls.predict_proba(rescaledfeat[s[1]])[:,1]
        metric = sklearn.metrics.roc_auc_score(Labels[s[1]], preds_proba)
        mymetrics.append(metric)
    
    return np.mean(mymetrics)

In [108]:
import warnings
warnings.filterwarnings('ignore', category=UserWarning, module='optuna')

sampler = optuna.samplers.TPESampler(multivariate=True, group=True, constant_liar=True, seed=42)
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=5000, timeout=120)

print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

print(study.best_params)

C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\optuna\samplers\_tpe\sampler.py:319: ExperimentalWarning: ``multivariate`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\optuna\samplers\_tpe\sampler.py:330: ExperimentalWarning: ``group`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
C:\Users\acer\AppData\Roaming\Python\Python312\site-packages\optuna\samplers\_tpe\sampler.py:338: ExperimentalWarning: ``constant_liar`` option is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2025-03-09 16:26:53,300] A new study created in memory with name: no-name-53855b86-20bf-4c73-adf4-8c99aae70470
[I 2025-03-09 16:26:54,336] Trial 0 finished with value: 0.7861190476190476 and parameters: {'n_estimators': 103, 'booster': 'gbtree', 'lambda': 3.968793330444372, 'alpha': 2.5361081166471375e-07, 'subsample'

Number of finished trials:  9
Best trial:
  Value: 0.8053333333333335
  Params: 
    n_estimators: 58
    booster: gbtree
    lambda: 8.505456981059524
    alpha: 0.18753546777666233
    subsample: 0.5543195836143536
    colsample_bytree: 0.07438075634395301
    max_depth: 7
    min_child_weight: 1
    eta: 0.13818852711527843
    gamma: 0.00020641342272342887
    grow_policy: lossguide
{'n_estimators': 58, 'booster': 'gbtree', 'lambda': 8.505456981059524, 'alpha': 0.18753546777666233, 'subsample': 0.5543195836143536, 'colsample_bytree': 0.07438075634395301, 'max_depth': 7, 'min_child_weight': 1, 'eta': 0.13818852711527843, 'gamma': 0.00020641342272342887, 'grow_policy': 'lossguide'}


The best trial from 9 trials found that using the following combination of hyperparameters achieved a model performance of 0.8053 (the value can represent accuracy, AUC, or other metric depending on the objective):

- 58 estimators (trees),
- Gradient Boosting Tree (gbtree),
- Regularization terms (lambda = 8.5, alpha = 0.19),
- Subsampling rate of 0.55 and colsample_bytree of 0.074,
- Tree depth of 7,
- Learning rate (eta) of 0.138,
- Tree-growing strategy: lossguide.

In [101]:
from copy import deepcopy

In [112]:
default_params = {"device": "cuda",
                  "verbosity": 0,
                  "objective": "binary:logistic",
                  "random_state": 42,
                  # use exact for small dataset.
                  "tree_method": "hist", # Specifies the tree construction algorithm (hist is a faster method than traditional exact methods).
                  "max_bin": 32}

all_params = deepcopy(study.best_params)
all_params.update(default_params)
"""The update function merges the default_params into all_params, meaning it updates or adds missing 
keys from default_params into all_params (which will now contain both the optimized and default hyperparameters)."""
print(all_params)

all_aucs = []
all_accuracies = []
all_f1s = []
all_precisions = []
all_recalls = []

for i in tqdm(range(200)):
    badsplit = True
    while badsplit:
        kf = KFold(n_splits=10, shuffle=True)
        splits = list(kf.split(rescaledfeat))
        badsplits = []
        for j in splits:
            if len(np.unique(Labels[j[1]])) == 2:
                badsplits.append(False)
            else:
                badsplits.append(True)
        badsplit = max(badsplits)
    aucs = []
    accuracies = []
    f1s = []
    precisions = []
    recalls = []
    for s in splits:
        cls = xgb.XGBClassifier(**all_params)
        cls.fit(rescaledfeat[s[0]], Labels[s[0]])
        preds_proba = cls.predict_proba(rescaledfeat[s[1]])[:,1]
        pred_labels = np.where(preds_proba > 0.5, 1, 0)
        auc = sklearn.metrics.roc_auc_score(Labels[s[1]], pred_labels)
        aucs.append(auc)
        accuracy = sklearn.metrics.accuracy_score(Labels[s[1]], pred_labels)
        accuracies.append(accuracy)
        f1 = sklearn.metrics.f1_score(Labels[s[1]], pred_labels)
        f1s.append(f1)
        precision = sklearn.metrics.precision_score(Labels[s[1]], pred_labels, zero_division= 0)
        precisions.append(precision)
        recall = sklearn.metrics.recall_score(Labels[s[1]], pred_labels)
        recalls.append(recall)
    all_aucs.append(np.mean(aucs))
    all_accuracies.append(np.mean(accuracies))
    all_f1s.append(np.mean(f1s))
    all_precisions.append(np.mean(precisions))
    all_recalls.append(np.mean(recalls))

{'n_estimators': 58, 'booster': 'gbtree', 'lambda': 8.505456981059524, 'alpha': 0.18753546777666233, 'subsample': 0.5543195836143536, 'colsample_bytree': 0.07438075634395301, 'max_depth': 7, 'min_child_weight': 1, 'eta': 0.13818852711527843, 'gamma': 0.00020641342272342887, 'grow_policy': 'lossguide', 'device': 'cuda', 'verbosity': 0, 'objective': 'binary:logistic', 'random_state': 42, 'tree_method': 'hist', 'max_bin': 32}


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [04:22<00:00,  1.31s/it]


In [114]:
best_acc_idx = np.argmax(all_accuracies)
print('accuracy: ' + str(all_accuracies[best_acc_idx]) + ', ' + str(all_accuracies_std[best_acc_idx]))
print('precision: ' + str(all_precisions[best_acc_idx]) + ', ' + str(all_precisions_std[best_acc_idx]))
print('recall: ' + str(all_recalls[best_acc_idx]) + ', ' + str(all_recalls_std[best_acc_idx]))
print('f1: ' + str(all_f1s[best_acc_idx]) + ', ' + str(all_f1s_std[best_acc_idx]))
print('auc: ' + str(all_aucs[best_acc_idx]) + ', ' + str(all_aucs_std[best_acc_idx]))

accuracy: 0.78, 0.07483314773547883
precision: 0.764047619047619, 0.20023583034787265
recall: 0.8280952380952382, 0.3205247433679826
f1: 0.7813553113553113, 0.22238680054660226
auc: 0.7863095238095238, 0.08707908152885684


In [123]:
print('auc: ' + str(np.mean(all_aucs)) + ', ' + str(np.mean(all_aucs_std)))
print('accuracy: ' + str(np.mean(all_accuracies)) + ', ' + str(np.mean(all_accuracies_std)))
print('f1: ' + str(np.mean(all_f1s)) + ', ' + str(np.mean(all_f1s_std)))
print('precision: ' + str(np.mean(all_precisions)) + ', ' + str(np.mean(all_precisions_std)))
print('recall: ' + str(np.mean(all_recalls)) + ', ' + str(np.mean(all_recalls_std)))

auc: 0.693750496031746, 0.12659207477317053
accuracy: 0.6842500000000001, 0.12645713487390203
f1: 0.6882307500506029, 0.19309984340375436
precision: 0.6705805555555555, 0.24229624626885482
recall: 0.760638492063492, 0.29467126057355786


* Tuned parameters show significant improvements across all metrics (AUC, accuracy, F1 score, precision, recall) compared to the default parameters.
* The AUC improvement suggests better classification ability with the tuned model.
* Accuracy, F1 score, precision, and recall improvements demonstrate that the tuned model is better at both distinguishing classes and making correct predictions, achieving a good balance between precision and recall.